In [1]:
#récupération de l'ap de bigml

from bigml.api import BigML
from pandas import read_csv
from sklearn.metrics import confusion_matrix
import pandas as pan
import matplotlib.pyplot as plt

In [2]:
#charge de l'api bigml pour pouvoir faire appel à ces fonctions

api = BigML(project='project/5d94a451eba31d46690001cf')

### récupération du fichier train full

In [3]:
#récupération du fichier csv bigml à traiter

train_full = api.create_source('./train_full.csv')

In [4]:
#test des sources

api.ok(train_full)

True

In [5]:
#création dataset depuis le fichier csv importer précédement

train_full_dataset = api.create_dataset(train_full)

### split du fichier train full 

In [6]:
#split du train full

train_dataset = api.create_dataset(
    train_full_dataset, {"name": "Train (80%)",
                     "sample_rate": 0.8, "seed": "Full train"})
validation_dataset = api.create_dataset(
    train_full_dataset, {"name": "Train (20%)",
                     "sample_rate": 0.8, "seed": "Full train",
                     "out_of_bag": True})

In [7]:
# fonction de calcul d'auc dépendant du % de train
def auc_ensemble(train_dataset, percent):
    train_split = api.create_dataset(
        train_dataset, {"name": f"Train_split ({percent}%)",
                     "sample_rate": percent/100, "seed": "train"})
    
    ensemble = api.create_ensemble([train_split],{"objective_field":"SeriousDlqin2yrs"})
    batch_prediction = api.create_batch_prediction(ensemble, train_split, {"all_fields":True, "probabilities":True, 'prediction_name':'Prediction'})
    api.ok(batch_prediction)
    api.download_batch_prediction(batch_prediction, filename='./predictions_split.csv')
    error = read_csv('./predictions_split.csv', index_col=0)
    error.to_csv('./predictions_split.csv', index=False)
    
    pos = error.loc[error['SeriousDlqin2yrs'] == 1] # Tous les Positif  
    neg = error.loc[error['SeriousDlqin2yrs'] == 0] # Tous les negatifs  
    x = 0 
    y = 0 # pour chaque 1 proba ( P ) dans toutes les 1 proba (P)
    for threshold_pos in pos['1 probability']: # pour chaque 1 Proba ( N ) dans toutes les 1 proba ( N )     
        for threshold_neg in neg['1 probability']:         
            if threshold_pos > threshold_neg:             
                x += 1          
            y += 1 
    return x/y


In [8]:
# fonction de calcul d'auc dépendant du % de train
def auc_deepnet(train_dataset, percent):
    train_split = api.create_dataset(
        train_dataset, {"name": f"Train_split ({percent}%)",
                     "sample_rate": percent/100, "seed": "train"})
    
    deepnet = api.create_deepnet([train_split],{"objective_field":"SeriousDlqin2yrs"})
    batch_prediction = api.create_batch_prediction(deepnet, train_split, {"all_fields":True, "probabilities":True, 'prediction_name':'Prediction'})
    api.ok(batch_prediction)
    api.download_batch_prediction(batch_prediction, filename='./predictions_split.csv')
    error = read_csv('./predictions_split.csv', index_col=0)
    error.to_csv('./predictions_split.csv', index=False)
    
    pos = error.loc[error['SeriousDlqin2yrs'] == 1] # Tous les Positif  
    neg = error.loc[error['SeriousDlqin2yrs'] == 0] # Tous les negatifs  
    x = 0 
    y = 0 # pour chaque 1 proba ( P ) dans toutes les 1 proba (P)
    for threshold_pos in pos['1 probability']: # pour chaque 1 Proba ( N ) dans toutes les 1 proba ( N )     
        for threshold_neg in neg['1 probability']:         
            if threshold_pos > threshold_neg:             
                x += 1          
            y += 1 
    return x/y


In [9]:
# affichage de la courbe de gzin en fonction de la valeur seuil
auc_ensemble = [auc_ensemble(train_dataset, n) for n in range(10,100,10)]
auc_deepnet = [auc_deepnet(train_dataset, n) for n in range(10,100,10)]
per = [(n) for n in range(10,100,10)]
plt.plot(per, [auc_ensemble, auc_deepnet])
plt.show()

KeyboardInterrupt: 

In [ ]:
auc_ensemble

In [ ]:
auc_deepnet